In [1]:
# ===========================
# Import Libraries
# ===========================

# Deep Learning & Torch Utilities
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, Subset

# Computer Vision & Image Processing
import cv2
from PIL import Image
import numpy as np

# Dataset & Transformations
import torchvision
from torchvision import transforms
from torchvision.datasets import ImageFolder
import albumentations as A
from albumentations.pytorch import ToTensorV2

# Segmentation Models
import segmentation_models_pytorch as smp

# Utilities
import os
from pathlib import Path
from tqdm import tqdm

/users/Etu3/21402333/.local/lib/python3.9/site-packages/albumentations/__init__.py:24: UserWarning: A new version of Albumentations is available: 1.4.24 (you have 1.4.22). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()


In [2]:
class CustomDataset(Dataset):
    def __init__(self, image_dir, mask_dir, transform=None):
        self.image_dir = image_dir
        self.mask_dir = mask_dir
        self.transform = transform
        self.images = os.listdir(image_dir)

    def __len__(self):
        return len(self.images)

    def __getitem__(self, index):
        img_path = os.path.join(self.image_dir, self.images[index])
        mask_path = os.path.join(self.mask_dir, self.images[index])
        image = np.array(Image.open(img_path).convert("RGB"))
        mask = np.array(Image.open(mask_path).convert("L"), dtype=np.float32)
        for i in range(mask.shape[0]):  # Parcourir les lignes
            for j in range(mask.shape[1]):  # Parcourir les colonnes
                if 127 < mask[i, j] :  # Si la valeur du pixel est 255
                    mask[i, j] = 1  # Remplace par 1
                else:
                    mask[i, j] = 0  # Remplace par 0


        if self.transform is not None:
            augmentations = self.transform(image=image, mask=mask)
            image = augmentations["image"]
            mask = augmentations["mask"]

        return image, mask

In [13]:
def save_checkpoint(state, filename="my_checkpoint.pth.tar"):
    print("=> Saving checkpoint")
    torch.save(state, filename)

def load_checkpoint(checkpoint, model):
    print("=> Loading checkpoint")
    model.load_state_dict(checkpoint["state_dict"])

def get_loaders(
    train_dir,
    train_maskdir,
    val_dir,
    val_maskdir,
    batch_size,
    train_transform,
    val_transform,
    num_workers=4,
    pin_memory=True,
):
    train_ds = CustomDataset(
        image_dir=train_dir,
        mask_dir=train_maskdir,
        transform=train_transform,
    )

    train_loader = DataLoader(
        train_ds,
        batch_size=batch_size,
        num_workers=num_workers,
        pin_memory=pin_memory,
        shuffle=True,
    )

    val_ds = CustomDataset(
        image_dir=val_dir,
        mask_dir=val_maskdir,
        transform=val_transform,
    )

    val_loader = DataLoader(
        val_ds,
        batch_size=batch_size,
        num_workers=num_workers,
        pin_memory=pin_memory,
        shuffle=False,
    )

    return train_loader, val_loader

def check_accuracy(loader, model, threshold=0.5, device="cuda"):
    """
    Évalue la précision et calcule le Dice Coefficient pour le modèle.
    
    Args:
        loader (DataLoader): DataLoader pour les données de validation/test.
        model (torch.nn.Module): Modèle à évaluer.
        threshold (float): Seuil pour binariser les prédictions.
        device (str): Appareil cible ("cuda" ou "cpu").
    """
    model.eval()
    num_correct = 0
    num_samples = 0
    dice_scores = []

    with torch.no_grad():
        for x, y in loader:
            # Convertir les tenseurs en FloatTensor et déplacer sur le GPU
            x = x.float().to(device)
            y = y.float().unsqueeze(1).to(device)

            # Prédictions
            preds, _ = model(x)

            # Calcul du Dice Coefficient
            preds_binarized = (torch.sigmoid(preds) > threshold).float()
            intersection = (preds_binarized * y).sum()
            union = preds_binarized.sum() + y.sum()
            dice = (2.0 * intersection) / (union + 1e-6)  # Ajout d'un epsilon pour éviter la division par zéro
            dice_scores.append(dice.item())

            # Binarisation des prédictions pour la précision
            num_correct += (preds_binarized == y).sum()
            num_samples += preds.numel()

    # Moyenne des scores de Dice
    avg_dice = sum(dice_scores) / len(dice_scores)
    acc = num_correct / num_samples

    print(f"Accuracy: {acc * 100:.2f}%, Average Dice: {avg_dice:.4f}")
    model.train()


def save_predictions_as_imgs(
    loader, model, folder="saved_images/", device="cuda"
):
    model.eval()
    for idx, (x, y) in enumerate(loader):
        x = x.to(device=device)
        with torch.no_grad():
            preds = torch.sigmoid(model(x))
            preds = (preds > 0.5).float()
        torchvision.utils.save_image(
            preds, f"{folder}/pred_{idx}.png"
        )
        torchvision.utils.save_image(y.unsqueeze(1), f"{folder}{idx}.png")

    model.train()

In [7]:
LEARNING_RATE = 1e-4
BATCH_SIZE = 32
NUM_WORKERS = 2
PIN_MEMORY = True
LOAD_MODEL = False
TRAIN_IMG_DIR = "./EBHI-SEG-Segmentation-train/image/"
TRAIN_MASK_DIR = "./EBHI-SEG-Segmentation-train/label/"
VAL_IMG_DIR = "./EBHI-SEG-Segmentation-test-for-eval/image/"
VAL_MASK_DIR = "./EBHI-SEG-Segmentation-test-for-eval/label/"


train_transform = A.Compose([
    A.Resize(256, 256),
    A.ElasticTransform(alpha=1.0, sigma=50, p=0.25),
    A.RandomBrightnessContrast(brightness_limit=0.2, contrast_limit=0.2, p=0.25),
    A.Rotate(limit=35, p=0.7),
    A.HorizontalFlip(p=0.5),
    A.VerticalFlip(p=0.1),
    A.GaussianBlur(blur_limit=(3, 7), p=0.15),
    ToTensorV2(),
])

val_transforms = A.Compose(
        [   A.Resize(256, 256),
            ToTensorV2(),
        ],
    )

train_loader, val_loader = get_loaders(
        TRAIN_IMG_DIR,
        TRAIN_MASK_DIR,
        VAL_IMG_DIR,
        VAL_MASK_DIR,
        BATCH_SIZE,
        train_transform,
        val_transforms,
        NUM_WORKERS,
        PIN_MEMORY,
    )

In [16]:
# Spécifier le GPU cible
# Spécifier le GPU cible
device = torch.device("cuda:3")
Resnet34 = torchvision.models.resnet34(pretrained=False)
weights_path = "resnet34_cerberus_torchvision .pth" 

# Charger les poids dans l'encodeur du modèle U-Net
state_dict = torch.load(weights_path, map_location=device)  # Charger les poids sauvegardés
Resnet34.load_state_dict(state_dict)  # Charger les poids dans l'encodeur
# Déplacer le modèle complet vers le GPU cible
Resnet34 = Resnet34.to(device)
# Afficher un résumé du modèle
print(Resnet34)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

/tmp/ipykernel_2058794/168585376.py:8: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(weights_path, map_location=device)  # Charger les poids sauvegar

In [17]:
class DecoderBlock(nn.Module):
    """Upscaling then double conv"""

    def __init__(self, conv_in_channels, conv_out_channels, up_in_channels=None, up_out_channels=None):
        super().__init__()
        """
        eg:
        decoder1:
        up_in_channels      : 1024,     up_out_channels     : 512
        conv_in_channels    : 1024,     conv_out_channels   : 512

        decoder5:
        up_in_channels      : 64,       up_out_channels     : 64
        conv_in_channels    : 128,      conv_out_channels   : 64
        """
        if up_in_channels==None:
            up_in_channels=conv_in_channels
        if up_out_channels==None:
            up_out_channels=conv_out_channels

        self.up = nn.ConvTranspose2d(up_in_channels, up_out_channels, kernel_size=2, stride=2)
        self.conv = nn.Sequential(
            nn.Conv2d(conv_in_channels, conv_out_channels, kernel_size=3, padding=1, bias=False),
            nn.BatchNorm2d(conv_out_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(conv_out_channels, conv_out_channels, kernel_size=3, padding=1, bias=False),
            nn.BatchNorm2d(conv_out_channels),
            nn.ReLU(inplace=True)
        )

    # x1-upconv , x2-downconv
    def forward(self, x1, x2):
        x1 = self.up(x1)
        x = torch.cat([x1, x2], dim=1)
        return self.conv(x)

class UnetResnet34(nn.Module):
    def __init__(self, num_classes=1, resnet34 = Resnet34, num_classes_classification = 6):
        super().__init__()
        filters = [64, 128, 256, 512]

        self.firstlayer = nn.Sequential(*list(resnet34.children())[:3])
        self.maxpool = list(resnet34.children())[3]
        self.encoder1 = resnet34.layer1
        self.encoder2 = resnet34.layer2
        self.encoder3 = resnet34.layer3
        self.encoder4 = resnet34.layer4

        self.bridge = nn.Sequential(
            nn.Conv2d(filters[3], filters[3]*2, kernel_size=3, padding=1, bias=False),
            nn.BatchNorm2d(filters[3]*2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2)
            
        )


        self.decoder1 = DecoderBlock(conv_in_channels=filters[3]*2, conv_out_channels=filters[3])
        self.decoder2 = DecoderBlock(conv_in_channels=filters[3], conv_out_channels=filters[2])
        self.decoder3 = DecoderBlock(conv_in_channels=filters[2], conv_out_channels=filters[1])
        self.decoder4 = DecoderBlock(conv_in_channels=filters[1], conv_out_channels=filters[0])
        self.decoder5 = DecoderBlock(
            conv_in_channels=filters[1], conv_out_channels=filters[0], up_in_channels=filters[0], up_out_channels=filters[0]
        )

        self.lastlayer = nn.Sequential(
            nn.ConvTranspose2d(in_channels=filters[0], out_channels=filters[0], kernel_size=2, stride=2),
            nn.Conv2d(filters[0], num_classes, kernel_size=3, padding=1, bias=False)
        )
        
        # Nouveau décodeur pour la classification
        self.classification_decoder = nn.Sequential(
            nn.AdaptiveAvgPool2d((1, 1)),  # Pooling global
            nn.Flatten(),
            nn.Linear(512, 1000),
            nn.ReLU(inplace=True),
            nn.Linear(1000, num_classes_classification)  # Classes de classification
        )
        
    
    def forward(self, x):
        e1 = self.firstlayer(x)
        maxe1 = self.maxpool(e1)
        e2 = self.encoder1(maxe1)
        e3 = self.encoder2(e2)
        e4 = self.encoder3(e3)
        e5 = self.encoder4(e4)
        
        c = self.bridge(e5)
        
        d1 = self.decoder1(c, e5)
        d2 = self.decoder2(d1, e4)
        d3 = self.decoder3(d2, e3)
        d4 = self.decoder4(d3, e2)
        d5 = self.decoder5(d4, e1)
        
        class_out = self.classification_decoder(e5)

        out = self.lastlayer(d5)

        return out, class_out

In [18]:
new_model = UnetResnet34(num_classes=1, resnet34 = Resnet34, num_classes_classification = 6)

In [19]:
# Geler les poids de l'encodeur (ResNet34)
for param in new_model.parameters() :
    param.requires_grad = False
# for param in new_model.classification_decoder.parameters() :
    #param.requires_grad = True

for param in new_model.bridge.parameters() :
    param.requires_grad = True
for param in new_model.decoder1.parameters() :
    param.requires_grad = True
for param in new_model.decoder2.parameters() :
    param.requires_grad = True
for param in new_model.decoder3.parameters() :
    param.requires_grad = True
for param in new_model.decoder4.parameters() :
    param.requires_grad = True
for param in new_model.decoder5.parameters() :
    param.requires_grad = True
for param in new_model.lastlayer.parameters() :
    param.requires_grad = True
    
    
for name, param in new_model.named_parameters():
    print(f"{name}: requires_grad={param.requires_grad}")

firstlayer.0.weight: requires_grad=False
firstlayer.1.weight: requires_grad=False
firstlayer.1.bias: requires_grad=False
encoder1.0.conv1.weight: requires_grad=False
encoder1.0.bn1.weight: requires_grad=False
encoder1.0.bn1.bias: requires_grad=False
encoder1.0.conv2.weight: requires_grad=False
encoder1.0.bn2.weight: requires_grad=False
encoder1.0.bn2.bias: requires_grad=False
encoder1.1.conv1.weight: requires_grad=False
encoder1.1.bn1.weight: requires_grad=False
encoder1.1.bn1.bias: requires_grad=False
encoder1.1.conv2.weight: requires_grad=False
encoder1.1.bn2.weight: requires_grad=False
encoder1.1.bn2.bias: requires_grad=False
encoder1.2.conv1.weight: requires_grad=False
encoder1.2.bn1.weight: requires_grad=False
encoder1.2.bn1.bias: requires_grad=False
encoder1.2.conv2.weight: requires_grad=False
encoder1.2.bn2.weight: requires_grad=False
encoder1.2.bn2.bias: requires_grad=False
encoder2.0.conv1.weight: requires_grad=False
encoder2.0.bn1.weight: requires_grad=False
encoder2.0.bn1.bi

# Avec de la data augmentation et scheduele lr 

In [20]:
# Passer le modèle sur le GPU
device = torch.device("cuda:3" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")
model=new_model.to(device)

# Définir les hyperparamètres
LEARNING_RATE = 1e-4
NUM_EPOCHS = 30

# Fonction de perte pour segmentation binaire
loss_fn = nn.BCEWithLogitsLoss()

# Optimiseur
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=15, gamma=0.1)

# Initialiser le scaler pour le calcul mixte (amp)
scaler = torch.cuda.amp.GradScaler()

# Boucle d'entraînement
for epoch in range(NUM_EPOCHS):
    print(f"Epoch [{epoch + 1}/{NUM_EPOCHS}]")
    model.train()
    train_loader, _ = get_loaders(
        TRAIN_IMG_DIR,
        TRAIN_MASK_DIR,
        VAL_IMG_DIR,
        VAL_MASK_DIR,
        BATCH_SIZE,
        train_transform,
        val_transforms,
        NUM_WORKERS,
        PIN_MEMORY,
    )
    loop = tqdm(train_loader, total=len(train_loader), desc="Training")

    for batch_idx, (data, targets) in enumerate(loop):
        # Charger les données sur GPU
        data = data.float().to(device)
        targets = targets.float().unsqueeze(1).to(device)  # Ajouter une dimension pour le canal de sortie

        # Forward pass avec AMP (Mixed Precision)
        with torch.cuda.amp.autocast():
            predictions, _ = model(data)
            loss = loss_fn(predictions, targets)

        # Backward pass
        optimizer.zero_grad()
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        # Mise à jour de la barre de progression
        loop.set_postfix(loss=loss.item())
    scheduler.step()     
    print(loss)
    # Évaluer la précision après chaque époque
    check_accuracy(val_loader, model, device=device)

/tmp/ipykernel_2058794/474051137.py:18: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler()


Using device: cuda:3
Epoch [1/30]


Training:   0%|                                                                                               | 0/56 [00:00<?, ?it/s]/tmp/ipykernel_2058794/474051137.py:43: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():
Training: 100%|██████████████████████████████████████████████████████████████████████████| 56/56 [00:20<00:00,  2.76it/s, loss=0.341]

tensor(0.3410, device='cuda:3',
       grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)


Accuracy: 90.16%, Average Dice: 0.9176
Epoch [2/30]


...

tensor(0.1610, device='cuda:3',
       grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)


Accuracy: 93.68%, Average Dice: 0.9493
Epoch [30/30]


Training: 100%|██████████████████████████████████████████████████████████████████████████| 56/56 [00:19<00:00,  2.87it/s, loss=0.196]

tensor(0.1961, device='cuda:3',
       grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)


Accuracy: 93.77%, Average Dice: 0.9502
